## 實戰演練三

請根據demo3.ipynb 內的描述，將深度學習模型匯出，並透過建立一 API 讓模型上線，使用者可以調用該API 輸入資料並得到預測結果


### 問題一：使用Keras 建立深度學習模型預測客戶是否流失

In [3]:
import pandas
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
df = pandas.read_csv('https://raw.githubusercontent.com/ywchiu/tibamedl/master/Data/Churn_Modelling.csv', header = 0 )
df = df.iloc[:,3:]
geography = pandas.get_dummies(df['Geography'])
del geography['France']
gender = pandas.get_dummies(df['Gender'])
del gender['Male']
df = pandas.concat([geography, gender, df], axis = 1)
del df['Gender']
del df['Geography']
X = df.iloc[:,:-1]
y = df.iloc[:,-1]
train_x, test_x, train_y, test_y = train_test_split(X,y, test_size = 0.3, random_state = 42)
sc = StandardScaler()
train_x = sc.fit_transform(train_x)
test_x = sc.transform(test_x)

### 問題二：請匯出深度學習模型至目錄夾sample_model

In [ ]:
model

In [ ]:
#@markdown 檢視答案
model.save('mymodel')
! mkdir -p /content/predict_model/1/
! mv /content/mymodel/* /content/predict_model/1/



### 問題三：請使用 Tensorflow Serving 佈署模型

In [ ]:
#@markdown 安裝Tensorflow Serving 
! echo "deb [arch=amd64] http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | sudo tee /etc/apt/sources.list.d/tensorflow-serving.list && \
curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | sudo apt-key add -
! apt-get update && apt-get install tensorflow-model-server  /dev/null

In [ ]:
tensorflow_model_server 

In [ ]:
#@markdown 檢視答案
%%bash --bg
nohup tensorflow_model_server \
  --rest_api_port=8501 \
  --model_name=saved_model \
  --model_base_path="/content/predict_model/" >server.log 2>&1